# Join nationalmaps demographic data, with latitude and longitude and .DAT sales data

In [6]:
from pathlib import Path
import geopandas as gpd
import pandas as pd


DATA_DIR = (Path().resolve().parent / "data" )
joined_data_path = DATA_DIR / "01_interim" / "nationalmaps" / "joined_nationalmaps.shp"
dat_with_lat_lon_path = DATA_DIR / "01_interim/valuergeneral/DAT_with_lat_lon.csv"

features_path = DATA_DIR / "02_processed" / "features_raw.csv"

In [7]:
df_natmaps = gpd.read_file(joined_data_path)
df_natmaps.shape

(2454, 71)

In [8]:
df_dat_with_lat_lon = pd.read_csv(dat_with_lat_lon_path)
df_dat_with_lat_lon.shape
df_dat_with_lat_lon[['propertyHouseNumber', 'lat','lon']]

,propertyHouseNumber,lat,lon
0,59,NaN,NaN
1,28,-34.349438,150.916366
2,808 C,-34.287014,150.948116
3,51,-34.436344,150.868383
4,16,-34.176149,150.992961
...,...,...,...
58253,NaN,NaN,NaN
58254,NaN,NaN,NaN
58255,NaN,NaN,NaN
58256,NaN,NaN,NaN


In [9]:
df_dat_with_lat_lon.columns

Index(['Unnamed: 0', 'index', 'fileType', 'districtCode', 'downloadDateTime',
       'submitterUserId', 'districtCode.1', 'propertyId', 'saleCounter',
       'downloadDateTime.1', 'propertyName', 'propertyUnitNumber',
       'propertyHouseNumber', 'propertyStreetName', 'propertyLocality',
       'propertyPostCode', 'area', 'areaType', 'contractDate',
       'settlementDate', 'purchasePrice', 'zoning', 'natureOfProperty',
       'primaryPurpose', 'strataLotNumber', 'componentCode', 'saleCode',
       'percentInterestOfSale', 'dealingNumber', 'districtCode.2',
       'propertyId.1', 'saleCounter.1', 'downloadDateTime.2',
       'propertyLegalDescription', 'districtCode.3', 'propertyId.2',
       'saleCounter.2', 'downloadDateTime.3', 'purchaserVendor',
       'totalRecords', 'totalBRecords', 'totalCRecords', 'totalDRecords',
       'address_str', 'address', 'lat', 'lon'],
      dtype='object')

### Demographic zones data mapped to addresses.

In [10]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

# Assuming G is your GeoPandas dataframe with a 'geometry' column of Polygons
# and A is your Pandas dataframe with 'lat' and 'lon' columns.
A = df_dat_with_lat_lon.copy()
G = df_natmaps.copy()

# Convert the 'lat' and 'lon' columns of A into a GeoSeries of Point geometries
geometry = [Point(xy) for xy in zip(A['lon'], A['lat'])]
A = gpd.GeoDataFrame(A, geometry=geometry)

# Ensure both dataframes use the same coordinate reference system (CRS)
A.set_crs(G.crs, inplace=True)

# Perform the spatial join; this associates rows of A with polygons in G
result = gpd.sjoin(A, G, how='left', op='within')

# 'result' now has all columns from A and G where each point in A falls within a polygon in G

### Exploring the joined data
- a single property Id with duplicate rows

In [43]:
result.propertyId[~pd.isna(result.propertyId)].value_counts()
c = ['fileType', 'downloadDateTime',
       'submitterUserId', 'propertyId', 'saleCounter',
       'downloadDateTime.1', 'area', 'areaType', 'contractDate',
       'settlementDate', 'purchasePrice', 'natureOfProperty',
       'primaryPurpose', 'componentCode', 'saleCode',
       'percentInterestOfSale', 'dealingNumber',
       'purchaserVendor',
       'totalRecords']
result.loc[result.propertyId==4400049.0,c].head(10)

,fileType,downloadDateTime,submitterUserId,propertyId,saleCounter,downloadDateTime.1,area,areaType,contractDate,settlementDate,purchasePrice,natureOfProperty,primaryPurpose,componentCode,saleCode,percentInterestOfSale,dealingNumber,purchaserVendor,totalRecords
10795,NaN,NaN,NaN,4400049.0,23.0,20220103 01:53,300.0,M,20191128.0,20211224.0,350000.0,V,VACANT LAND,AUS,NaN,0.0,AR772661,V,NaN
10796,NaN,NaN,NaN,4400049.0,24.0,20220103 01:53,300.0,M,20201027.0,20211224.0,412000.0,V,VACANT LAND,AUS,NaN,0.0,AR772662,P,NaN
11471,NaN,NaN,NaN,4400049.0,23.0,20220103 01:53,300.0,M,20191128.0,20211224.0,350000.0,V,VACANT LAND,AUS,NaN,0.0,AR772661,V,NaN
11472,NaN,NaN,NaN,4400049.0,24.0,20220103 01:53,300.0,M,20201027.0,20211224.0,412000.0,V,VACANT LAND,AUS,NaN,0.0,AR772662,P,NaN
12149,NaN,NaN,NaN,4400049.0,23.0,20220103 01:53,300.0,M,20191128.0,20211224.0,350000.0,V,VACANT LAND,AUS,NaN,0.0,AR772661,V,NaN
12150,NaN,NaN,NaN,4400049.0,24.0,20220103 01:53,300.0,M,20201027.0,20211224.0,412000.0,V,VACANT LAND,AUS,NaN,0.0,AR772662,P,NaN
12841,NaN,NaN,NaN,4400049.0,23.0,20220103 01:53,300.0,M,20191128.0,20211224.0,350000.0,V,VACANT LAND,AUS,NaN,0.0,AR772661,V,NaN
12842,NaN,NaN,NaN,4400049.0,24.0,20220103 01:53,300.0,M,20201027.0,20211224.0,412000.0,V,VACANT LAND,AUS,NaN,0.0,AR772662,P,NaN
13575,NaN,NaN,NaN,4400049.0,23.0,20220103 01:53,300.0,M,20191128.0,20211224.0,350000.0,V,VACANT LAND,AUS,NaN,0.0,AR772661,V,NaN
13576,NaN,NaN,NaN,4400049.0,24.0,20220103 01:53,300.0,M,20201027.0,20211224.0,412000.0,V,VACANT LAND,AUS,NaN,0.0,AR772662,P,NaN


## Select variables to create feature set from

In [11]:
import itertools

chunk_size = 10
c = list(result.columns)
chunks = [c[i:i + chunk_size] for i in range(0, len(c), chunk_size)]
# use itertools to iterate over sequential chunks of size 10 in the list
for chunk in chunks:
    print(chunk)

['Unnamed: 0', 'index', 'fileType', 'districtCode', 'downloadDateTime', 'submitterUserId', 'districtCode.1', 'propertyId', 'saleCounter', 'downloadDateTime.1']
['propertyName', 'propertyUnitNumber', 'propertyHouseNumber', 'propertyStreetName', 'propertyLocality', 'propertyPostCode', 'area', 'areaType', 'contractDate', 'settlementDate']
['purchasePrice', 'zoning', 'natureOfProperty', 'primaryPurpose', 'strataLotNumber', 'componentCode', 'saleCode', 'percentInterestOfSale', 'dealingNumber', 'districtCode.2']
['propertyId.1', 'saleCounter.1', 'downloadDateTime.2', 'propertyLegalDescription', 'districtCode.3', 'propertyId.2', 'saleCounter.2', 'downloadDateTime.3', 'purchaserVendor', 'totalRecords']
['totalBRecords', 'totalCRecords', 'totalDRecords', 'address_str', 'address', 'lat', 'lon', 'geometry', 'index_right', 'STE_CODE21']
['STE_NAME21', 'GCC_CODE21', 'GCC_NAME21', 'SA4_CODE21', 'SA4_NAME21', 'SA3_CODE21', 'SA3_NAME21', 'SA2_CODE21', 'SA2_NAME21', 'Males']
['Females', 'Persons', 'Sex

In [12]:
df_joined = pd.DataFrame(result)

### Filter out unwanted columns

In [13]:
f_pop_age = lambda c: (c.startswith('P') or c.startswith('M') or c.startswith('F')) and ('_' in c) and c[1].isdigit() or ('Median_age' in c)
f_income = lambda c: c in ['OBS_VALUE']
f_pop = lambda c: c in ['Females', 'Males', 'Persons', 'Sex_ratio']
f_gis = lambda c: c in ['lat', 'lon'] + ['STE_CODE21', 'STE_NAME21', 'GCC_CODE21', 'GCC_NAME21', 'SA4_CODE21', 'SA4_NAME21', 'SA3_CODE21', 'SA3_NAME21', 'SA2_CODE21', 'SA2_NAME21']
f_purchase_metadata = lambda c: c in ['purchasePrice','settlementDate', 'natureOfProperty', 'primaryPurpose', 'propertyUnitNumber', 'propertyHouseNumber', 'saleCounter']
col_filters = [f_pop_age, 
               f_income, 
               f_pop, 
               f_gis,
               f_purchase_metadata]
# new dataframe with only columns that match the filter
df_joined = df_joined[[c for f in col_filters for c in df_joined.columns if f(c)]]
df_joined.head()

,Median_age,M0_4,M5_9,M10_14,M15_19,M20_24,M25_29,M30_34,M35_39,M40_44,...,SA3_NAME21,SA2_CODE21,SA2_NAME21,saleCounter,propertyUnitNumber,propertyHouseNumber,settlementDate,purchasePrice,natureOfProperty,primaryPurpose
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,20,59,20211223.0,352000.0,R,RESIDENCE
1,41.5,636.0,710.0,774.0,696.0,618.0,408.0,522.0,610.0,722.0,...,Wollongong,107041150.0,Woonona - Bulli - Russell Vale,2.0,NaN,28,20211223.0,2510000.0,R,RESIDENCE
2,43.4,349.0,430.0,457.0,414.0,303.0,234.0,284.0,344.0,403.0,...,Wollongong,107041148.0,Thirroul - Austinmer - Coalcliff,3.0,NaN,808 C,20211223.0,1750000.0,V,VACANT LAND
3,33.4,429.0,453.0,376.0,500.0,1124.0,1037.0,720.0,571.0,459.0,...,Wollongong,107041549.0,Wollongong - West,4.0,NaN,51,20211223.0,1200000.0,R,RESIDENCE
4,38.6,338.0,333.0,331.0,310.0,284.0,219.0,272.0,308.0,301.0,...,Wollongong,107041147.0,Helensburgh,5.0,NaN,16,20211224.0,650000.0,R,RESIDENCE


In [14]:
df_joined.columns

Index(['Median_age', 'M0_4', 'M5_9', 'M10_14', 'M15_19', 'M20_24', 'M25_29',
       'M30_34', 'M35_39', 'M40_44', 'M45_49', 'M50_54', 'M55_59', 'M60_64',
       'M65_69', 'M70_74', 'M75_79', 'M80_84', 'M85_and_ov', 'F0_4', 'F5_9',
       'F10_14', 'F15_19', 'F20_24', 'F25_29', 'F30_34', 'F35_39', 'F40_44',
       'F45_49', 'F50_54', 'F55_59', 'F60_64', 'F65_69', 'F70_74', 'F75_79',
       'F80_84', 'F85_and_ov', 'P0_4', 'P5_9', 'P10_14', 'P15_19', 'P20_24',
       'P25_29', 'P30_34', 'P35_39', 'P40_44', 'P45_49', 'P50_54', 'P55_59',
       'P60_64', 'P65_69', 'P70_74', 'P75_79', 'P80_84', 'P85_and_ov',
       'OBS_VALUE', 'Males', 'Females', 'Persons', 'Sex_ratio', 'lat', 'lon',
       'STE_CODE21', 'STE_NAME21', 'GCC_CODE21', 'GCC_NAME21', 'SA4_CODE21',
       'SA4_NAME21', 'SA3_CODE21', 'SA3_NAME21', 'SA2_CODE21', 'SA2_NAME21',
       'saleCounter', 'propertyUnitNumber', 'propertyHouseNumber',
       'settlementDate', 'purchasePrice', 'natureOfProperty',
       'primaryPurpose'],
   

In [15]:
df_joined.to_csv(features_path, index=False)

In [16]:
df_joined.info()

<class 'pandas.core.frame.DataFrame'>
Index: 58258 entries, 0 to 58257
Data columns (total 79 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Median_age           15196 non-null  float64
 1   M0_4                 15196 non-null  float64
 2   M5_9                 15196 non-null  float64
 3   M10_14               15196 non-null  float64
 4   M15_19               15196 non-null  float64
 5   M20_24               15196 non-null  float64
 6   M25_29               15196 non-null  float64
 7   M30_34               15196 non-null  float64
 8   M35_39               15196 non-null  float64
 9   M40_44               15196 non-null  float64
 10  M45_49               15196 non-null  float64
 11  M50_54               15196 non-null  float64
 12  M55_59               15196 non-null  float64
 13  M60_64               15196 non-null  float64
 14  M65_69               15196 non-null  float64
 15  M70_74               15196 non-null  floa